In [1]:
# Import all relevant libraries
import warnings
def fxn(): 
	warnings.warn("deprecated",DeprecationWarning)

with warnings.catch_warnings( ):
    warnings.simplefilter("ignore")
    fxn( )

# Keras imports
import keras
from keras.models import Sequential, Model
from keras.layers import Input, UpSampling1D, BatchNormalization, MaxPooling1D, MaxPooling2D, Permute, Flatten, Softmax, Dense, Dropout, Conv1D, Conv2D, Conv2DTranspose, AveragePooling1D, AveragePooling2D, Activation, Reshape
from keras.utils import np_utils
from keras.optimizers import Adam

# Other
import numpy as np
import h5py
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Load data from specific trial
def get_trial(trial_num):    
    trial = h5py.File('../data/A0' + str(trial_num) + 'T_slice.mat', 'r')
    X = np.copy(trial['image'])
    y = np.copy(trial['type'])
    y = y[0,0:X.shape[0]:1]
    y = np.asarray(y, dtype=np.int32)
    y -= 769                            # shift class labels to [0-3]
    X = np.nan_to_num(X)[:, :22, :]     # remove EOG channels
    return X, y

def get_all_trials():
    X_total = np.concatenate([get_trial(trial_num)[0] for trial_num in range(1, 10)], axis=0)
    y_total = np.concatenate([get_trial(trial_num)[1] for trial_num in range(1, 10)], axis=0)
    return X_total, y_total

def stratified_CV_split(X, y, k):
    ''' Returns a stratified train/test split, for k number of splits. Split amount
    is based on k (equal splits).
    Return value is in the form [(train indices, test indices), ... for k folds ]
    '''
    skf = StratifiedKFold(n_splits=k)
    return skf.split(X, y)

def stratified_train_test_split(X, y, k, num_trials):
    ''' Returns a stratified train/test split, for k number of splits. Test size is 
    50 * number of trials X is composed of.
    Return value is in the form [(train indices, test indices), ... for k folds ]
    '''
    sss = StratifiedShuffleSplit(n_splits=k, test_size=50*num_trials)
    return sss.split(X, y)

In [3]:
# Get the data from all the people and generate train/test split
X, y = get_all_trials()
y_cat = keras.utils.to_categorical(y, num_classes=4)
#tt_splits = stratified_train_test_split(X, y, 5)
X_train, X_test, y_train, y_test = train_test_split(X, y )

one_hot_train = keras.utils.to_categorical(y_train)
one_hot_test = keras.utils.to_categorical(y_test)


# The data for each trial is of the shape (288, 22, 1000)
#   There are 288 samples per trial (12 of each class per "run", 4 classes, 6 "runs" 
#                                   at different time periods of the day)
#   There are 22 electrodes from the EEG (represents spatial aspect of the signals)
#   There are 1000 time units (4 seconds of data, sampled at 250Hz). The first 250 units
#                                   are when no movement occurs (but the cue is heard) and
#                                   the next 750 units are when the movement occurs
# The labels for each trial belong in one of 4 classes
#   0 - left
#   1 - right
#   2 - foot
#   3 - tongue

In [14]:
# Deep Implementation CNN

model = Sequential()

# Conv Pool Block 1
model.add(Conv2D(filters=25, kernel_size=(10,1), input_shape=(1000, 22, 1), kernel_initializer = 'glorot_normal', strides=1, data_format="channels_last"))
model.add(Conv2D(filters=25, kernel_size=(1,22), kernel_initializer = 'glorot_normal' ))
model.add(BatchNormalization())
model.add(Activation(activation = 'elu'))
model.add(MaxPooling2D(pool_size = (3,1), strides = (3,1)))
model.add(Dropout(0.5))
print(model.output_shape)



# Conv Pool Block 2
model.add(Conv2D(filters = 50, kernel_size = (10,1), activation = 'elu', kernel_initializer = 'glorot_normal'))
model.add(BatchNormalization())
model.add(Activation(activation = 'elu'))
model.add(MaxPooling2D(pool_size = (3,1), strides = (3,1)))
model.add(Dropout(0.5))
print(model.output_shape)

# Conv Pool Block 3
model.add(Conv2D(filters = 100, kernel_size = (10,1), activation = 'elu', kernel_initializer = 'glorot_normal'))
model.add(BatchNormalization())
model.add(Activation(activation = 'elu'))
model.add(MaxPooling2D(pool_size = (3,1), strides = (3,1)))
model.add(Dropout(0.5))
print(model.output_shape)

# Conv Pool Block 4
model.add(Conv2D(filters = 200, kernel_size = (10,1), activation = 'elu', kernel_initializer = 'glorot_normal'))
model.add(BatchNormalization())
model.add(Activation(activation = 'elu'))
model.add(MaxPooling2D(pool_size = (3,1), strides = (3,1)))
model.add(Dropout(0.5))
print(model.output_shape)

# Dense Layers
model.add(Flatten())
model.add(Dense(units=4, kernel_initializer='glorot_normal', activation = 'softmax'))
print(model.output_shape)


#Adam = Adam(lr=0.15)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


print(X_train.shape)
x_train = X_train.transpose((0,2,1))
print(x_train.shape)

x_test = X_test.transpose((0,2,1))


print(X_train.shape)
print(x_train.shape)



model.fit(x_train[:, :, :, None] , one_hot_train, epochs=150, batch_size=32, validation_data = (x_test[:, :, :, None], one_hot_test))


(None, 330, 1, 25)
(None, 107, 1, 50)
(None, 32, 1, 100)
(None, 7, 1, 200)
(None, 4)
(1944, 22, 1000)
(1944, 1000, 22)
(1944, 22, 1000)
(1944, 1000, 22)
Train on 1944 samples, validate on 648 samples
Epoch 1/150
1944/1944 [==============================] - 25s 13ms/step - loss: 2.3864 - acc: 0.2613 - val_loss: 1.4869 - val_acc: 0.2994
Epoch 2/150
1944/1944 [==============================] - 28s 14ms/step - loss: 1.9567 - acc: 0.3097 - val_loss: 1.3802 - val_acc: 0.3519
Epoch 3/150
1944/1944 [==============================] - 27s 14ms/step - loss: 1.7762 - acc: 0.3380 - val_loss: 1.3284 - val_acc: 0.3843
Epoch 4/150
1944/1944 [==============================] - 27s 14ms/step - loss: 1.5997 - acc: 0.3657 - val_loss: 1.2727 - val_acc: 0.4090
Epoch 5/150
1944/1944 [==============================] - 27s 14ms/step - loss: 1.5483 - acc: 0.3827 - val_loss: 1.2385 - val_acc: 0.4352
Epoch 6/150
1944/1944 [==============================] - 28s 14ms/step - loss: 1.4829 - acc: 0.4120 - val_loss: 1.1

Epoch 59/150
1944/1944 [==============================] - 29s 15ms/step - loss: 0.5520 - acc: 0.7855 - val_loss: 0.9045 - val_acc: 0.6543
Epoch 60/150
1944/1944 [==============================] - 28s 14ms/step - loss: 0.5308 - acc: 0.8035 - val_loss: 0.7600 - val_acc: 0.6821
Epoch 61/150
1944/1944 [==============================] - 26s 13ms/step - loss: 0.5351 - acc: 0.7989 - val_loss: 0.7477 - val_acc: 0.6960
Epoch 62/150
1944/1944 [==============================] - 26s 13ms/step - loss: 0.5449 - acc: 0.7819 - val_loss: 0.7563 - val_acc: 0.7006
Epoch 63/150
1944/1944 [==============================] - 27s 14ms/step - loss: 0.5554 - acc: 0.7855 - val_loss: 0.8150 - val_acc: 0.6728
Epoch 64/150
1944/1944 [==============================] - 28s 15ms/step - loss: 0.4988 - acc: 0.7963 - val_loss: 0.7568 - val_acc: 0.7114
Epoch 65/150
1944/1944 [==============================] - 28s 14ms/step - loss: 0.5341 - acc: 0.7968 - val_loss: 0.8499 - val_acc: 0.6914
Epoch 66/150
1944/1944 [==========

1944/1944 [==============================] - 31s 16ms/step - loss: 0.3029 - acc: 0.8915 - val_loss: 0.8799 - val_acc: 0.6852
Epoch 119/150
1944/1944 [==============================] - 27s 14ms/step - loss: 0.3012 - acc: 0.8904 - val_loss: 0.8976 - val_acc: 0.6806
Epoch 120/150
1944/1944 [==============================] - 27s 14ms/step - loss: 0.2805 - acc: 0.8956 - val_loss: 0.9281 - val_acc: 0.6806
Epoch 121/150
1944/1944 [==============================] - 26s 14ms/step - loss: 0.2636 - acc: 0.8945 - val_loss: 0.9514 - val_acc: 0.6759
Epoch 122/150
1944/1944 [==============================] - 26s 14ms/step - loss: 0.2757 - acc: 0.8935 - val_loss: 0.8896 - val_acc: 0.6744
Epoch 123/150
1944/1944 [==============================] - 26s 13ms/step - loss: 0.2566 - acc: 0.9043 - val_loss: 0.9789 - val_acc: 0.6821
Epoch 124/150
1944/1944 [==============================] - 27s 14ms/step - loss: 0.2717 - acc: 0.9012 - val_loss: 0.9289 - val_acc: 0.6836
Epoch 125/150
1944/1944 [================

ValueError: Error when checking input: expected conv2d_26_input to have 4 dimensions, but got array with shape (648, 1000, 22)

In [16]:
score = model.evaluate(x_test[:, :, :, None], one_hot_test, verbose=0)

In [17]:
print(score)

[0.96995184818903601, 0.68672839506172845]
